# Drowsiness Detection

In [1]:
# Importing necessary Libraries

from   scipy.spatial import distance as dist
from   imutils import face_utils
import imutils
import dlib
import cv2
import winsound

In [2]:
# Importing necessary Algorithms

detector  = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [3]:
# Getting the Co-ordinates for the Left and Right Eye

(lstart,lend) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rstart,rend) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [4]:
# Creating an User-defined function to determine Eye_Aspect_Ratio

def eyeAspectRatio(eye):
    # Vertical
    a   = dist.euclidean(eye[1], eye[5])
    b   = dist.euclidean(eye[2], eye[4])   
    # Horizontal
    c   = dist.euclidean(eye[0], eye[3])   
    ear = (a+b)/(2*c)
    return ear

In [14]:
# Detecting the Drowsiness

count    =  0

camera = cv2.VideoCapture(0)
while True:
    _, frame  = camera.read()
    frame     = cv2.flip(frame,1)
    frame     = imutils.resize(frame, width=450)
    gray      = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    rects     = detector(gray,0)
    for rect in rects:
        shape = predictor(gray,rect)
        shape = face_utils.shape_to_np(shape)
        
        lefteye  = shape[lstart:lend]
        righteye = shape[rstart:rend]
       
        
        # Drawing the Contours
        cv2.drawContours(frame,[cv2.convexHull(lefteye)],-1,(0,0,255),1)
        cv2.drawContours(frame,[cv2.convexHull(righteye)],-1,(0,0,255),1)
        
         
        leftear  = eyeAspectRatio(lefteye)
        rightear = eyeAspectRatio(righteye)
        
        ear      = (leftear+rightear) / 2.0
        
        if ear < 0.3:
            count +=1
            
            if count >= 48:
                cv2.putText(frame,"Drowsiness Detection",(10,30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255),2)
                winsound.Beep(2500,1000)
        
        else:
            count = 0
        
        
    
    cv2.imshow("frame",frame)
    if cv2.waitKey(1) == ord("q"):
        break
        
camera.release()
cv2.destroyAllWindows()